In [99]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference.models import JsonSchemaFormat, ChatCompletionsToolDefinition, FunctionDefinition
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage

from dotenv import load_dotenv

load_dotenv(override=True)

client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_DEEPSEEK_ENDPOINT"],
    credential=AzureKeyCredential(os.environ["AZURE_DEEPSEEK_API_KEY"]),
)

In [79]:
def search_engine(query: str):
    return "Search results"

# search_engine_tool = ChatCompletionsToolDefinition(function=FunctionDefinition(
#     name="search_engine",
#     description="Search the web for information",
#     parameters={
#         "type": "object",
#         "properties": {
#             "query": {"type": "string", "description": "The query to search the web for"}
#         },
#         "required": ["query"]
#     }
# ))

MOCK_SEARCH_ENGINE_PROMPT = """You are a search engine results mocker. Given a query, mock the top 5 search engine results. Each result should be a mock article paragraph. You don't have to think or reason, just generate random results. Format your mock as
```search 1
mock article paragraph
```
```search 2
mock article paragraph
```
...
```"""

def mock_search_engine(query: str):
    response_gen = client.complete(
    messages=[
        SystemMessage(content=MOCK_SEARCH_ENGINE_PROMPT),
        UserMessage(content=query),
    ],
    model="Analysis-POC-DeepSeek-R1",
    stream=True,
    # tools=[search_engine_tool]
    )
    
    begin_response = False
    
    full_response = ""
    for token in response_gen:
        if len(token["choices"]) > 0:
            tok = token["choices"][0]['delta']['content']
            if begin_response:
                full_response += tok
            print(tok, end="", flush=True)
            if tok.lower() == "</think>":
                begin_response = True
    return full_response

query_results = mock_search_engine("Nvidia's latest news")

<think>
Okay, I need to generate mock search results for "Nvidia's latest news". Let me think of plausible topics related to Nvidia. They're big in GPUs, AI, acquisitions, partnerships, earnings reports, and product launches. I should create five different snippets.

First result: Maybe a new GPU announcement. Let's say the RTX 5090 with specs and release date. Second result: AI advancements, like a collaboration with a tech company. Third: Acquisition news, perhaps buying an AI startup. Fourth: Earnings report beating expectations. Fifth: A new AI platform for healthcare. Each should be a paragraph, concise and varied. Check for different angles to cover various areas Nvidia is involved in. Make sure they look realistic but fictional.
</think>

```search 1  
Nvidia unveils its next-gen **RTX 5090 GPU**, boasting a 70% performance leap over the RTX 4090. Early benchmarks suggest groundbreaking ray-tracing capabilities and AI-driven upscaling, with a Q1 2025 release window. Analysts pre

In [81]:
SYSTEM_PROMPT = """You are a helpful assistant that has access to a search engine. Think about the user's request and decide if you need to search the web. If you have a query, wrap it around
```query
```

For example, if the user asks "What is flights are available from US?", you should search the web for
```query
flights from US
```

You will continue digging and searching the web until you have enough information to give a comprehensive answer to the user's question.
"""

def assistant(query: str):

    response_gen = client.complete(
        messages=[
            SystemMessage(content=SYSTEM_PROMPT),
            UserMessage(content="Is Dogecoin a good investment?"),
        ],
        model="Analysis-POC-DeepSeek-R1",
        stream=True,
        # tools=[search_engine_tool]
    )

    for token in response_gen:
        if len(token["choices"]) > 0:
            print(token["choices"][0]['delta']['content'], end="", flush=True)
            # print(token)

<think>
Okay, the user wants to know if Dogecoin's price will go up. I need to provide a helpful answer. Let me start by recalling that cryptocurrency prices are highly volatile and influenced by many factors.

First, I should mention the main factors that affect Dogecoin's price. Market sentiment is a big one. Since Dogecoin started as a meme, its price often reacts to social media trends and celebrity endorsements, like Elon Musk's tweets. 

Then there's adoption and utility. If more businesses start accepting Dogecoin as payment, that could drive demand. I should check if there's any recent news about companies adopting it. Maybe search for recent partnerships or integrations.

Also, the overall crypto market trends are important. If Bitcoin and Ethereum are up, altcoins like Dogecoin might follow. I need to look at the current market trends. Are we in a bull or bear market? Recent price charts would help here. 

Regulatory changes can impact all cryptocurrencies. Have there been an

In [102]:
import re

SYSTEM_PROMPT = """You are a helpful assistant that has access to a search engine. Think about the user's request and decide if you need to search the web. If you have a query, wrap it around
```query
```

For example, if the user asks "What is flights are available from US?", you should search the web for
```query
flights from US
```

You will ONLY HAVE 1 QUERY AT A TIME. After you respond, user will reply with the query results. You will then continue digging and searching the web until you have enough information to give a comprehensive answer to the user's question. When responding, ONLY respond with the query. If you are done with your research, respond with a full report wrapped in ```report``` tags. During report writing, you DON'T NEED to write any query.
"""

class DeepseekClient:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint=os.environ["AZURE_DEEPSEEK_ENDPOINT"],
            credential=AzureKeyCredential(os.environ["AZURE_DEEPSEEK_API_KEY"]),
        )
        
    def complete(self, messages: list, verbose: bool = False, ignore_think: bool = False):
        response_gen = self.client.complete(
            messages=messages,
            model="Analysis-POC-DeepSeek-R1",
            stream=True,
        )

        full_response = ""
        end_think = False
        for token in response_gen:
            if len(token["choices"]) > 0:
                tok = token["choices"][0]['delta']['content']
                if verbose:
                    print(tok, end="", flush=True)
                if tok.lower() == "</think>":
                    end_think = True
                if not ignore_think:
                    full_response += tok
                else:
                    if end_think:
                        full_response += tok
        return full_response



class ResearchAgent:
    def __init__(self):
        self.client = DeepseekClient()
        self.messages = [SystemMessage(content=SYSTEM_PROMPT)]

    def search(self, query: str):
        self.messages.append(UserMessage(content=query))

        response = self.client.complete(
            messages=self.messages,
            verbose=True,
            ignore_think=True,
        )

        return response
    def extract_query_content(self, text):
        match = re.search(r"```query\n(.*?)\n```", text, re.DOTALL)
        return match.group(1) if match else None
    
    def start(self, query: str):
        while True:
            response = self.search(query)
            if "```report" in response:
                return response
            query = self.extract_query_content(response)
            self.messages.append(AssistantMessage(content=query))
            query_results = mock_search_engine(query)
            self.messages.append(UserMessage(content=query_results))

agent = ResearchAgent()
report = agent.start("Is Dogecoin a good investment?")
# response = agent.start("Is Dogecoin a good investment?")

<think>
Okay, the user is asking if Dogecoin is a good investment. Let me start by understanding what Dogecoin is. From what I remember, Dogecoin started as a meme cryptocurrency but has gained some popularity. I should check its current market status, like price, market cap, and recent trends.

I need to look into factors that make an investment good. Volatility is a big one with cryptocurrencies. Dogecoin might be highly volatile, so that's a risk. Also, its adoption rate – are more places accepting it as payment? Maybe major companies like Tesla or others?

Another aspect is the technology behind Dogecoin. How does it compare to Bitcoin or Ethereum in terms of blockchain technology, transaction speed, and fees? If it's not as advanced, that could affect long-term viability.

Market sentiment is important too. Influencers like Elon Musk have tweeted about Dogecoin before, impacting its price. I should check if there's ongoing influential support or if that's waned.

Regulatory enviro

KeyboardInterrupt: 

In [84]:
import re

def extract_query_content(text):
    match = re.search(r"```query\n(.*?)\n```", text, re.DOTALL)
    return match.group(1) if match else None

# Example usage
text = """
Some text before
```query
SELECT * FROM users WHERE age > 25;
```
Some text after
"""

print(extract_query_content(query_results))


None


In [85]:
print(query_results)



```search 1  
Nvidia unveils its next-gen **RTX 5090 GPU**, boasting a 70% performance leap over the RTX 4090. Early benchmarks suggest groundbreaking ray-tracing capabilities and AI-driven upscaling, with a Q1 2025 release window. Analysts predict shortages due to unprecedented demand from gamers and AI researchers.  
```  

```search 2  
Nvidia partners with **Tesla** to power its Full Self-Driving (FSD) systems with custom AI chips. The collaboration aims to achieve "Level 5 autonomy" by 2026, leveraging Nvidia’s Blackwell architecture for real-time decision-making. Musk calls it a "game-changer for robotics and transportation."  
```  

```search 3  
Nvidia acquires **Boston Dynamics** in a surprise $12B deal, signaling a push into robotics and embodied AI. Critics question the valuation, but CEO Jensen Huang argues the merger will accelerate "AI-powered physical systems" for logistics and healthcare. Spot the robot now runs on Omniverse software.  
```  

```search 4  
Nvidia’s 

In [105]:
import re

text = "<query>What is the weather in Tokyo?</query>"

matches = re.findall(r'<query>(.*?)</query>', text, re.DOTALL)
query = matches[0].strip() if matches else None

In [106]:
query

'What is the weather in Tokyo?'